### Código funcional (espera interação do usuário pelo teclado)

In [ ]:
import os
import pandas as pd
import streamlit as st
import google.generativeai as genai
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent, AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms import OpenAI
from langchain_community.callbacks.manager import get_openai_callback
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
import time
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect
from pydantic import BaseModel
from getpass import getpass



def search_responses_csv(query):
    csv_file_path = "responses.csv"
    if os.path.isfile(csv_file_path):
        df = pd.read_csv(csv_file_path)
        matching_rows = df[df['query'].str.contains(query, case=False, na=False)]
        if not matching_rows.empty:
            return matching_rows.iloc[0]['response']
    return None

def setup_environment_and_agent():
    # Get the directory of the current script
    #script_dir = os.path.dirname(os.path.abspath(__file__))
    script_dir = os.getcwd()  # Obtém o diretório de trabalho atual
    env_file = 'inf.env'
    env_path = os.path.join(script_dir, env_file)  # Use the script's directory

    if not os.path.exists(env_path):
        print(f"Error: '{env_file}' not found at {env_path}. Creating a new one.")
        with open(env_path, 'w') as f:
            f.write("# Add your OpenAI API key and DB credentials here\n")
            f.write("OPENAI_API_KEY=\n")
            f.write("DB_USER=\n")
            f.write("DB_PASSWORD=\n")
            f.write("DB_HOST=\n")
            f.write("DB_NAME=\n")
            f.write("DB_PORT=5432\n")
        print(f"Please edit '{env_path}' with your credentials and rerun the script.")
        exit(1)

    if not load_dotenv(env_path):
        print(f"Warning: Failed to load environment variables from '{env_path}'.")

    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        openai_api_key = getpass("Enter your OpenAI API key: ")
        os.environ["OPENAI_API_KEY"] = openai_api_key
        with open(env_path, 'a') as f:
            f.write(f"OPENAI_API_KEY={openai_api_key}\n")

    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_name = os.getenv("DB_NAME")
    db_port = os.getenv("DB_PORT", "5432")

    SQLALCHEMY_DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
    engine = create_engine(SQLALCHEMY_DATABASE_URL)
    db = SQLDatabase.from_uri(SQLALCHEMY_DATABASE_URL, schema="tarifas")

    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)

    csv_tool = Tool(
        name="SearchResponsesCSV",
        func=search_responses_csv,
        description="Search for previous responses in the 'responses.csv' file based on the user's query."
    )

    prompt_template = """
    Seu objetivo é auxiliar os usuários com suas consultas. Você tem acesso a duas fontes principais de informação:
    1. O banco de dados SQL 'sql_bot' para consultas relacionadas a tarifas e cálculos.
    2. O arquivo 'responses.csv', que contém respostas previamente geradas para consultas anteriores.
    **Instruções gerais**:
    - Primeiro, verifique se a consulta do usuário já foi respondida no 'responses.csv' usando a ferramenta 'SearchResponsesCSV'. Se encontrar uma resposta correspondente, retorne-a diretamente ao usuário.
    - Se não houver resposta no 'responses.csv', processe a consulta normalmente usando o banco de dados SQL ou gere uma resposta conversacional, conforme apropriado.
    - Para consultas relacionadas ao banco de dados, forneça respostas detalhadas em formato tabular. Se o resultado puder ser grande, limite a um subconjunto (ex.: 10 primeiros registros) e informe o usuário.
    - Para consultas não relacionadas ao banco de dados, responda de forma amigável e conversacional.
    - Não realize alterações no banco de dados, apenas consultas (SELECT).
    **Cálculos específicos**:
    - Para cálculos como tarifas médias ou yield, use sempre a fórmula da média ponderada deflacionada pelo IPCA, conforme exemplos abaixo.
    - Exemplo de cálculo de tarifa média para janeiro de 2024:
    SELECT nm_empresa,
    SUM(nr_assentos * nr_tarifa * ipca_base.nr_ipca / ft.nr_ipca) / SUM(nr_assentos) AS tarifa_media_ponderada_deflacionada
    FROM "tarifas"."ft_tarifas_domesticas" ft
    CROSS JOIN (SELECT ipca AS nr_ipca
    FROM "tarifas"."dm_ipca"
    ORDER BY nr_ano_mes_referencia DESC
    LIMIT 1) AS ipca_base
    WHERE ft.nr_ano_referencia = 2024 AND ft.nr_mes_referencia = 1
    GROUP BY nm_empresa

    - Exemplo de cálculo de yield para janeiro de 2024:
    SELECT nm_empresa,
    SUM(nr_tarifa * nr_ipca_base * nr_assentos / nr_ipca) / SUM(nr_distancia * nr_assentos) AS yield
    FROM "tarifas"."ft_tarifas_domesticas" ft
    CROSS JOIN (SELECT nr_ipca AS nr_ipca_base
    FROM "tarifas"."ft_tarifas_domesticas"
    ORDER BY nr_ano_referencia DESC, nr_mes_referencia DESC
    LIMIT 1) AS ipca_base
    WHERE ft.nr_ano_referencia = 2024 AND ft.nr_mes_referencia = 1
    GROUP BY nm_empresa

    **Regras adicionais**:
    - Não informe valores absolutos, sempre a média ponderada, assim se for socitado a tarifa, ou o yield, retorno tarifas e yields podenrados médios, então lembre da divisão pelos assentos!
    - Considere a GOL como VRG Linhas Aéreas, GOL LINHAS AÉREAS S.A., ou GLO.
    - Para destino-s, priorize sg_icao_origem e sg_icao_destino; se não encontrar, use sg_iata_origem e sg_iata_destino.
    - Se o usuario perguntar sobre uma cidade ou um estado, lembre de fazer a reuniao dos aeroportos da cidade ou estado.
    - Nunca altere a lógica do CROSS JOIN para o IPCA mais recente.
    - A menos que explicitamete solicitado, sempre calcule tarifas e yields médios do setor e separado por empresa.
    - Quando perguntado qual a tarifa ou yield, SEMPRE considere o valor médio. NÃO há interesse em saber o valor absoluto DE TARIFA E YIELD.
    - Se perguntado sobre a GOl, considere ser a GOL LINHAS AÉREAS S.A. (GLO na coluna sg_empresa_icao) Se pergutando sobre a TAM considere ser a TAM LINHAS AÉREAS S.A. (TAM na coluna sg_empresa_icao) e se perguntado sobre a AZUL, considere ser a AZUL LINHAS AÉREAS BRASILEIRAS S/A (AZU na coluna sg_empresa_icao) 
    """
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        extra_tools=[csv_tool],
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        system_message=prompt_template
    )
    return agent_executor, prompt_template

def save_response(query, response, memory):
    csv_file_path = "responses.csv"
    data = {
        "query": [query],
        "response": [response],
        "memory": [memory]
    }
    df = pd.DataFrame(data)
    if not os.path.isfile(csv_file_path):
        df.to_csv(csv_file_path, index=False)
    else:
        df.to_csv(csv_file_path, mode='a', header=False, index=False)

def count_tokens(chain, query, prompt_template):
    with get_openai_callback() as cb:
        full_query = prompt_template + "\nUser query: " + query
        try:
            result = chain.run(full_query)
            print(f'Spent a total of {cb} tokens')
        except Exception as e:
            result = "I don't have sufficient resources to process your query!"
            return e, full_query
    return result, full_query

# Configuração do agente
agent_executor, prompt_template = setup_environment_and_agent()

# Função para rodar no terminal
def run_terminal():
    print("Assistente Conversacional de Tarifas Domésticas")
    print("Digite sua consulta abaixo (ou 'exit' para sair):")
    
    while True:
        user_query = input("Consulta: ")
        if user_query.lower() == 'exit':
            print("Encerrando o programa. Até logo!")
            break
        elif user_query:
            print("Processando sua consulta...")
            response, memory = count_tokens(agent_executor, user_query, prompt_template)
            print("\nResposta do Bot:")
            print(response)
            save_choice = input("\nA resposta está correta? (s/n): ").lower()
            if save_choice == 's':
                save_response(user_query, response, memory)
                print("Resposta salva com sucesso em 'responses.csv'!")
            else:
                print("Resposta descartada.")

# Função para rodar no Streamlit
def run_streamlit():
    st.title("Assistente Conversacional de Tarifas Domésticas")
    st.write("Digite sua consulta abaixo e receba uma resposta do bot. Você pode salvar respostas aprovadas no arquivo 'responses.csv'.")

    user_query = st.text_input("Digite sua consulta (ou 'exit' para sair):", "")
    
    if st.button("Enviar Consulta"):
        if user_query.lower() == 'exit':
            st.write("Encerrando o aplicativo. Até logo!")
        elif user_query:
            with st.spinner("Processando sua consulta..."):
                response, memory = count_tokens(agent_executor, user_query, prompt_template)
            st.write("**Resposta do Bot:**")
            st.write(response)

            st.session_state['last_query'] = user_query
            st.session_state['last_response'] = response
            st.session_state['last_memory'] = memory
            st.session_state['show_confirmation'] = True

    if 'show_confirmation' in st.session_state and st.session_state['show_confirmation']:
        st.write("**A resposta está correta?**")
        col1, col2 = st.columns(2)
        with col1:
            if st.button("Sim, salvar"):
                save_response(st.session_state['last_query'], st.session_state['last_response'], st.session_state['last_memory'])
                st.success("Resposta salva com sucesso em 'responses.csv'!")
                st.session_state['show_confirmation'] = False
        with col2:
            if st.button("Não, descartar"):
                st.warning("Resposta descartada.")
                st.session_state['show_confirmation'] = False

# Menu de escolha
def main():
    print("Escolha como deseja executar o programa:")
    print("1 - Rodar no terminal")
    print("2 - Rodar com Streamlit")
    
    escolha = input("Digite sua escolha (1 ou 2): ")
    
    if escolha == "1":
        run_terminal()
    elif escolha == "2":
        # Salva o código Streamlit em um arquivo temporário
        with open("temp_app.py", "w") as f:
            streamlit_code = f"""
import os
import pandas as pd
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.agents import create_sql_agent, AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_community.callbacks.manager import get_openai_callback
from langchain.agents import Tool
from dotenv import load_dotenv
from sqlalchemy import create_engine
from getpass import getpass

def search_responses_csv(query):
    csv_file_path = "responses.csv"
    if os.path.isfile(csv_file_path):
        df = pd.read_csv(csv_file_path)
        matching_rows = df[df['query'].str.contains(query, case=False, na=False)]
        if not matching_rows.empty:
            return matching_rows.iloc[0]['response']
    return None

def setup_environment_and_agent():
    env_file = 'inf.env'
    env_path = os.path.join(os.path.dirname(__file__), env_file)
    if not os.path.exists(env_path):
        st.error(f"'inf.env' not found. Please create it with your credentials.")
        return None, None
    load_dotenv(env_path)
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        openai_api_key = getpass("Enter your OpenAI API key: ")
        os.environ["OPENAI_API_KEY"] = openai_api_key
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_name = os.getenv("DB_NAME")
    db_port = os.getenv("DB_PORT", "5432")
    SQLALCHEMY_DATABASE_URL = f"postgresql://{{db_user}}:{{db_password}}@{{db_host}}:{{db_port}}/{{db_name}}"
    engine = create_engine(SQLALCHEMY_DATABASE_URL)
    db = SQLDatabase.from_uri(SQLALCHEMY_DATABASE_URL, schema="tarifas")
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    csv_tool = Tool(
        name="SearchResponsesCSV",
        func=search_responses_csv,
        description="Search for previous responses in the 'responses.csv' file."
    )
    prompt_template = \"\"\"{prompt_template}\"\"\"
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        extra_tools=[csv_tool],
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        system_message=prompt_template
    )
    return agent_executor, prompt_template

def save_response(query, response, memory):
    csv_file_path = "responses.csv"
    data = {{"query": [query], "response": [response], "memory": [memory]}}
    df = pd.DataFrame(data)
    if not os.path.isfile(csv_file_path):
        df.to_csv(csv_file_path, index=False)
    else:
        df.to_csv(csv_file_path, mode='a', header=False, index=False)

def count_tokens(chain, query, prompt_template):
    with get_openai_callback() as cb:
        full_query = prompt_template + "\\nUser query: " + query
        try:
            result = chain.run(full_query)
            st.text(f'Spent a total of {{cb}} tokens')
        except Exception as e:
            result = "I don't have sufficient resources to process your query!"
            return e, full_query
    return result, full_query

agent_executor, prompt_template = setup_environment_and_agent()

if agent_executor:
    st.title("Assistente GEAC - IA")
    st.write("Digite sua consulta abaixo e receba uma resposta do bot.")
    user_query = st.text_input("Digite sua consulta (ou 'exit' para sair):", "")
    if st.button("Enviar Consulta"):
        if user_query.lower() == 'exit':
            st.write("Encerrando o aplicativo. Até logo!")
        elif user_query:
            with st.spinner("Processando sua consulta..."):
                response, memory = count_tokens(agent_executor, user_query, prompt_template)
            st.write("**Resposta do Bot:**")
            st.write(response)
            st.session_state['last_query'] = user_query
            st.session_state['last_response'] = response
            st.session_state['last_memory'] = memory
            st.session_state['show_confirmation'] = True
    if 'show_confirmation' in st.session_state and st.session_state['show_confirmation']:
        st.write("**A resposta está correta?**")
        col1, col2 = st.columns(2)
        with col1:
            if st.button("Sim, salvar"):
                save_response(st.session_state['last_query'], st.session_state['last_response'], st.session_state['last_memory'])
                st.success("Resposta salva com sucesso em 'responses.csv'!")
                st.session_state['show_confirmation'] = False
        with col2:
            if st.button("Não, descartar"):
                st.warning("Resposta descartada.")
                st.session_state['show_confirmation'] = False
"""
            # Escreve o código no arquivo com o prompt_template escapado
            f.write(streamlit_code.replace("{prompt_template}", prompt_template.replace('"', '\\"')))
        # Executa o Streamlit
        os.system("streamlit run temp_app.py")
    else:
        print("Escolha inválida! Por favor, digite 1 ou 2.")

if __name__ == "__main__":
    main()

Escolha como deseja executar o programa:
1 - Rodar no terminal
2 - Rodar com Streamlit


Digite sua escolha (1 ou 2):  2
